In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as fn
from awsglue.dynamicframe import DynamicFrame    
glueContext = GlueContext(SparkContext.getOrCreate())

In [ ]:
btweets = glueContext.create_dynamic_frame.from_catalog(database="ai_driven_social_media_dashboard", table_name="custom2020")

In [ ]:
btweets.printSchema()

In [ ]:
print("Count: " + str(btweets.count()))

In [ ]:
customDF = btweets.toDF()    

In [ ]:
customDF = customDF.withColumn('date', fn.concat(fn.lit("2020"),fn.lit("-"),fn.col("partition_0"),fn.lit("-"),fn.col("partition_1")))    

In [ ]:
customDF = customDF.withColumn('time', fn.concat(fn.col("partition_2"),fn.lit(":"),fn.lit("00:00.000"))) 

In [ ]:
customDF = customDF.withColumn('final', fn.concat(fn.col("date"),fn.lit(" "),fn.col("time"))) 

In [ ]:
customDF.printSchema()

In [ ]:
customDF.show(10)

In [ ]:
customDF= customDF.withColumn('final',fn.to_date(customDF.final))

In [ ]:
customDF= customDF.withColumn('final',fn.to_timestamp((customDF.final),"yyyy-MM-dd HH:mm:ss.SSS"))

In [ ]:
customDF.show(10)

In [ ]:
customDF = customDF.drop('date','time')    

In [ ]:
customDF.show(10)

In [ ]:
customDynamicFrame = DynamicFrame.fromDF(customDF, glueContext, "customDF_df")

In [ ]:
datasink4 = glueContext.write_dynamic_frame.from_options(frame = customDynamicFrame, connection_type = "s3", connection_options = {"path": "s3://bt-tweet-tweetsbucket-13nlui8dbzj6k/Custom2020/parquet/"}, format = "parquet", transformation_ctx = "datasink4")    